# Setup do modelo

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Instalar bibliotecas
!pip install transformers torch fastapi uvicorn nest-asyncio unsloth bitsandbytes

In [32]:
!pip install -U bitsandbytes

In [33]:
!pip install pyngrok

In [34]:
!pip install flask-ngrok

In [35]:
# Importar bibliotecas
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
import nest_asyncio
import uvicorn
from unsloth import FastLanguageModel
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware

In [36]:
# Iniciar o FastAPI
app = FastAPI()

# Configurar as origens permitidas
origins = [
    "*"  # Permite requisições de qualquer origem. Para produção, especifique os domínios permitidos.
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # Permite as origens definidas acima
    allow_credentials=True,
    allow_methods=["*"],    # Permite todos os métodos (GET, POST, etc.)
    allow_headers=["*"],    # Permite todos os cabeçalhos
)

# Constantes do modelo
START_TAG = "<|start_header_id|>assistant<|end_header_id|>\n\n"
END_TAG = "<|eot_id|>"

In [37]:
!ngrok authtoken "TOKEN"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok_tunnel = ngrok.connect(8005)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

# Execução do modelo

In [39]:
# Configuração do modelo
max_seq_length = 240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [40]:
# Salvando classes do modelo
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/ModelosM11/BrastelAssistantNLG", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    use_safetensors=True,
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-13' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=1024, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=1024, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

# API

In [41]:
# Definir o modelo da solicitação
class QueryModel(BaseModel):
    intent: str
    question: str

In [45]:
# Rota principal para geração de respostas
@app.post("/generate")

async def generate_response(query: QueryModel):
    messages = [{"role": "system",  # Instruções do sistema para o bot
"content": """Você é um bot de atendimento da Brastel Remit.
Abaixo está uma Intenção que descreve a classificação de uma pergunta,
seguida de uma pergunta que deve ser respondida.
Escreva uma resposta que atenda adequadamente à solicitação,
com foco nas operações, procedimentos e serviços oferecidos pela Brastel."""
},{"role": "user", "content": f"Intenção: {query.intent}" + f"\nPergunta: {query.question}"}
    ]

    # Preparar o input para o modelo
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Gerar a resposta
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    outputs = model.generate(
        input_ids=inputs,
        streamer=text_streamer,
        max_new_tokens=128,
        use_cache=True,
        temperature=1.5,
        min_p=0.1
    )

    # A resposta é impressa pelo text_streamer, mas podemos capturá-la e retornar
    # Para capturar a resposta sem exibi-la diretamente, podemos usar o método `generate` sem o streamer
    response_ids = model.generate(
        input_ids=inputs,
        max_new_tokens=248,
        use_cache=True,
        temperature=0.5,
        min_p=0.1
    )

    # Convertendo os ids gerados de volta para texto
    predicted_text = tokenizer.decode(response_ids[0]).split(START_TAG)[1].split(END_TAG)[0].strip()

    print(query.intent)

    # Retornar a resposta gerada como JSON
    return {"message": predicted_text}


# Executar a aplicação no servidor uvicorn (necessário para rodar a API)
# Isso precisa ser executado manualmente no Colab, pois o servidor precisa rodar continuamente
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, port=8005)


INFO:     Started server process [3476]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8005 (Press CTRL+C to quit)


Olá! O Brasil! Por favor, é só enviar a sua senhora para que o valor seja enviado no prazo de 2 minutos, inclusive nos fins de semana e feriados!<|eot_id|>
INFO:     189.201.201.2:0 - "POST /generate HTTP/1.1" 200 OK
A taxa de envio do Brastel para o Japão está em ¥9,000<|eot_id|>
INFO:     177.92.77.170:0 - "POST /generate HTTP/1.1" 200 OK
Você pode fazer a remessa através do aplicativo Brastel Remit, ou via bancos parceiros no Brasil.<|eot_id|>
INFO:     189.201.201.2:0 - "POST /generate HTTP/1.1" 200 OK
Bom dia! Para enviar dinheiro para o Brasil, você deve ser cliente da nossa aplicação e registrar todos os dados do beneficiário. Assim que for validado o dados, pode registrar os dados do beneficiário na aplicação e você poderá fazer o pagamento por aplicativo. Assim vamos validar seus dados e você já vai poder fazer a remessa. Com o nosso serviço você irá economizar passo a passo e teremos a transação enviada no mesmo dia!<|eot_id|>
INFO:     177.92.77.170:0 - "POST /generate HTTP/

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3476]
